In [1]:
from sklearn.metrics import mean_squared_error as MSE
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from os import walk
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.FATAL)
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
from math import floor
import pandas as pd
from tqdm import tqdm
from helperFunctions import *
import helperFunctions
import statistics
import cv2
import pytesseract
import regex
import shutil

In [2]:
companies = get_companies()
companiesWeighted, weight = get_companies_weighted()

In [3]:
def company(stock,startDate, endDate,startingCapital,plot = False, s = 1,final = False):
    model = "Best/Models/{}-model.json".format(stock)

    start = startDate
    
   
    variablesToInclude = ['close', 'volume',"rsi", "adx", "fastd", "fastk", "macd"]
    
    realStockPrice, data, newTrain,numberOfFeatures = get_data(stock,startDate,endDate,variablesToInclude)
    if not numberOfFeatures:
        return 0,0,0,0,0
    trainingSet = newTrain.iloc[:,0:numberOfFeatures].values #convert to numpy to train RNN
    ySet = data.astype(float).values.reshape(-1, 1)

    # ## Feature Scaling

    # Use normalization x - min(x) / max(min) - min(x)
    sc = MinMaxScaler(feature_range=(0,1)) # all values between 0 and 1
    ySC = MinMaxScaler(feature_range=(0,1))
    _ = ySC.fit_transform(ySet)
    _ = sc.fit_transform(trainingSet)

    regressor=tf.keras.models.load_model(model)
    
    realStockPrice,_,newTest,numberOfFeatures = get_data(stock,startDate,endDate,variablesToInclude) #convert to numpy to train RNN
    
    xTest = convert_to_numpy(newTest, numberOfFeatures, sc)
    
    if not len(xTest):
        return 0,0,0,0,0
    
    predictedPrice, realStockPrice = get_predictions(regressor,xTest,realStockPrice, ySC, s)
    # %% [markdown]
    # ## Predict price

    profitSeries, dailyReturns, loss, profit, buy, sold = calculate_profit(
        startingCapital, predictedPrice, realStockPrice)
    
    
    if plot:
        plot_data(stock, realStockPrice, predictedPrice, buy, profitSeries, loss, final = final)

    
    sharperatio, sortinoRatio, calmerRatio = calculate_ratios(dailyReturns, start, endDate)
    
    

    return profit,sharperatio,sortinoRatio,calmerRatio,sold


In [4]:
def eval(startDate="1982-3-12", endDate="2022-02-1",weighted = False, companies = companies, s = 1):
    startDate = datetime.strptime(str(startDate), "%Y-%m-%d")
    endDate = datetime.strptime(str(endDate), "%Y-%m-%d")
    startDateThreshold = datetime.strptime("1982-3-12", "%Y-%m-%d")
    spstart = startDate

    if startDateThreshold > startDate:
        spstart = startDateThreshold
    profit = 0

    with hidden_prints():
        sp500 = yf.download('^GSPC',spstart,endDate)

    try:
        profitsp500percentage = ((sp500['Close'][-1] - sp500['Close'][0])/sp500['Close'][0])*100
        profitsp500 = (500_000) * (profitsp500percentage/100)
    except:
        profitsp500percentage = -1
        profitsp500 = -1
    profit = 0
    sharperatio = []
    sortinoRatio = []
    calmerRatio = []
    profits = {}

    topStock = ""
    maxProfit = float('-inf')
    lowestStock = ""
    minProfit = float('inf')

    index = 0
    sold = 0
    value = 500_000

    for i in tqdm(companies):
        #with HiddenPrints():
        index += 1
    

        if weighted:    
            weightC = weight[i]
            startingCapital = 500_000 * weightC
            profitC, sharperatioC, sortinoRatioC, calmerRatioC,soldC = company(i, startDate, endDate, startingCapital, s = s,final=True,plot=False)

        else:
            i = i[:-4]
            print(i)
            profitC, sharperatioC, sortinoRatioC, calmerRatioC, soldC = company(
                i, startDate, endDate,s = s)

        profit += profitC
        profits[profitC] = i
        sharperatio.append(sharperatioC)
        sortinoRatio.append(sortinoRatioC)
        calmerRatio.append(calmerRatioC)
        sold += soldC
        if profitC < minProfit:
            minProfit = profitC
            lowestStock = i

        if profitC > maxProfit:
            maxProfit = profitC
            topStock = i
    
    sharperatio = pd.Series(sharperatio).dropna()
    sortinoRatio = pd.Series(sortinoRatio).dropna()
    
    profitPercentage = (profit/(value))*100
    profit = round(profit,2)
    profitPercentage = round(profitPercentage,2)
    profitsp500 = round(profitsp500,2)
    profitsp500percentage = round(profitsp500percentage,2)
    sharperatio = round((sharperatio.mean()),2)
    sortinoRatio = round((sortinoRatio.mean()),2)
    calmerRatio = 0
    minProfit = round(minProfit,2)
    maxProfit = round(maxProfit,2)

    if profitsp500 < profit:
        beat = True
    else:
        beat = False
        

    print("Start Date: {}".format(str(startDate.date())))
    print("End Date: {}".format(str(endDate.date())))
    print("Profit: ${}".format(profit))
    print("ROI: {}%".format(profitPercentage))
    print("Most Profitable Stock: {}".format(topStock))
    print("Profit for {}: ${}".format(topStock,maxProfit))
    print("Least Profitable Stock: {}".format(lowestStock))
    print("Profit for {}: ${}".format(lowestStock,minProfit))
    print("Profit S&P500: ${}".format(profitsp500))
    print("S&P500 ROI: {}%".format(profitsp500percentage))
    print("Sharpe Ratio: {}".format(sharperatio))
    print("Sortino Ratio: {}".format(sortinoRatio))
    print("Calmer Ratio: {}".format(calmerRatio))
    print("Traded {} Times".format(sold))
    return profit, beat


In [ ]:
eval(startDate='2021-01-01', endDate='2021-03-1', s=3, weighted=True, companies=companiesWeighted)

100%|██████████| 59/59 [03:25<00:00,  3.49s/it]

Start Date: 2021-01-01
End Date: 2021-03-01
Profit: $24218.14
ROI: 4.84%
Most Profitable Stock: NXPI
Profit for NXPI: $5408.34
Least Profitable Stock: AEP
Profit for AEP: $-55.97
Profit S&P500: $14929.81
S&P500 ROI: 2.99%
Sharpe Ratio: 13.38
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 127 Times


(24218.14, True)

In [14]:
eval(startDate='2018-10-01', endDate='2019-01-1',s=3, weighted=True, companies=companiesWeighted)

100%|██████████| 59/59 [03:36<00:00,  3.67s/it]

Start Date: 2018-10-01
End Date: 2019-01-01
Profit: $-45060.91
ROI: -9.01%
Most Profitable Stock: CRM
Profit for CRM: $3922.86
Least Profitable Stock: NXPI
Profit for NXPI: $-9166.22
Profit S&P500: $-71418.55
S&P500 ROI: -14.28%
Sharpe Ratio: 6.69
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 250 Times


(-45060.91, True)

In [15]:
eval(startDate='2021-03-01', endDate='2021-05-1',
     s=3, weighted=True, companies=companiesWeighted)

100%|██████████| 59/59 [03:25<00:00,  3.49s/it]

Start Date: 2021-03-01
End Date: 2021-05-01
Profit: $28416.27
ROI: 5.68%
Most Profitable Stock: NXPI
Profit for NXPI: $10625.9
Least Profitable Stock: TWTR
Profit for TWTR: $-615.37
Profit S&P500: $35797.38
S&P500 ROI: 7.16%
Sharpe Ratio: 19.05
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 150 Times


(28416.27, False)

In [16]:
eval(startDate='2021-05-01', endDate='2021-07-1',
     s=3, weighted=True, companies=companiesWeighted)

100%|██████████| 59/59 [03:41<00:00,  3.76s/it]

Start Date: 2021-05-01
End Date: 2021-07-01
Profit: $13705.58
ROI: 2.74%
Most Profitable Stock: RCL
Profit for RCL: $3503.53
Least Profitable Stock: DISH
Profit for DISH: $-1446.94
Profit S&P500: $12502.78
S&P500 ROI: 2.5%
Sharpe Ratio: 12.91
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 189 Times


(13705.58, True)

In [17]:
eval(startDate='2021-07-01', endDate='2021-09-1',
     s=3, weighted=True, companies=companiesWeighted)


100%|██████████| 59/59 [03:45<00:00,  3.82s/it]

Start Date: 2021-07-01
End Date: 2021-09-01
Profit: $2677.3
ROI: 0.54%
Most Profitable Stock: MS
Profit for MS: $1715.48
Least Profitable Stock: GPS
Profit for GPS: $-5258.57
Profit S&P500: $23465.63
S&P500 ROI: 4.69%
Sharpe Ratio: 16.27
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 165 Times


(2677.3, False)

In [18]:

eval(startDate='2021-09-01', endDate='2021-11-1',
     s=3, weighted=True, companies=companiesWeighted)


100%|██████████| 59/59 [03:54<00:00,  3.98s/it]

Start Date: 2021-09-01
End Date: 2021-11-01
Profit: $7075.48
ROI: 1.42%
Most Profitable Stock: IVZ
Profit for IVZ: $2301.37
Least Profitable Stock: GPS
Profit for GPS: $-5411.69
Profit S&P500: $8984.13
S&P500 ROI: 1.8%
Sharpe Ratio: 13.55
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 212 Times


(7075.48, False)

In [19]:

eval(startDate='2021-11-01', endDate='2022-01-1',
     s=3, weighted=True, companies=companiesWeighted)


100%|██████████| 59/59 [03:52<00:00,  3.93s/it]

Start Date: 2021-11-01
End Date: 2022-01-01
Profit: $1718.52
ROI: 0.34%
Most Profitable Stock: NXPI
Profit for NXPI: $5800.62
Least Profitable Stock: GPS
Profit for GPS: $-7241.43
Profit S&P500: $16528.08
S&P500 ROI: 3.31%
Sharpe Ratio: 13.49
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 231 Times


(1718.52, False)

In [5]:
startDate = datetime.strptime('2021-01-01', "%Y-%m-%d")
endDate = datetime.strptime('2021-03-01', "%Y-%m-%d")
profitC, sharperatioC, sortinoRatioC, calmerRatioC, soldC = company(
    'UAL', startDate=startDate, endDate=endDate, plot=True, s=3,startingCapital=500,final=True)
profitC

57.35031573637605

<Figure size 432x288 with 0 Axes>

In [6]:
eval(startDate='1982-03-01', endDate='2022-02-01',
     s=3, weighted=True, companies=companiesWeighted)

  0%|          | 0/59 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [8]:
totalWins = 0
epochs = 10

datesEval = []
with hidden_prints():
    for i in tqdm(range(epochs)):
        startDate, endDate = get_dates()
        datesEval.append(startDate)
        _, win = eval(startDate = startDate, endDate = endDate, s = 3,weighted=True, companies=companiesWeighted)
        if win:
            totalWins += 1

if totalWins > 5:
    print(f"You beat S&P500 {totalWins} times out of {epochs} times.")
else:
    print(f"S&P500 beat you {epochs-totalWins} times out of {epochs} times.")

print(f'Evaluvated on dates {datesEval}')

100%|██████████| 10/10 [29:56<00:00, 179.69s/it]

S&P500 beat you 6 times out of 10 times.
Evaluvated on dates ['2019-10-03', '2020-07-17', '2020-12-26', '2020-12-18', '2019-05-30', '2019-04-04', '2022-06-13', '2022-02-11', '2020-01-07', '2019-11-21']


In [5]:
eval(startDate='2022-02-14', endDate='2022-04-08',
     s=3, weighted=True, companies=companiesWeighted)

100%|██████████| 59/59 [03:53<00:00,  3.96s/it]

Start Date: 2022-02-14
End Date: 2022-04-08
Profit: $13837.96
ROI: 2.77%
Most Profitable Stock: DISH
Profit for DISH: $3828.53
Least Profitable Stock: GPS
Profit for GPS: $-3886.05
Profit S&P500: $11193.48
S&P500 ROI: 2.24%
Sharpe Ratio: 5778882538321512.0
Sortino Ratio: 0.0
Calmer Ratio: 0
Traded 242 Times


(13837.96, True)

In [6]:
for week in range(8):
    profits = {}
    for c in companiesWeighted:
        img = cv2.imread(f'Final/Week{week}/{c}-Week{week}.png')
        text = pytesseract.image_to_string(img)
        profitC = float(regex.search(
            "([$¢]\d+.\d+)|(\$0)|([$¢]-\d+.\d+)", text).group(0)[1:])
        profits[c] = profitC
    maxCompany = max(profits, key=profits.get)
    maxProfit = profits[maxCompany]
    minCompany = min(profits, key=profits.get)
    minProfit = profits[minCompany]
    medProfit = statistics.median(profits.values())
    medCompany = list(profits.keys())[list(profits.values()).index(medProfit)]
    print(f"Week{week}")
    print(f"Max Profit:${maxProfit} - {maxCompany}")
    print(f"Min Profit:${minProfit} - {minCompany}")
    print(f"Med Profit:${medProfit} - {medCompany}")
    shutil.copyfile(f"Final/Week{week}/{maxCompany}-Week{week}.png", f"Week{week}/Max-{maxCompany}.png")
    shutil.copyfile(f"Final/Week{week}/{medCompany}-Week{week}.png", f"Week{week}/Med-{medCompany}.png")
    shutil.copyfile(f"Final/Week{week}/{minCompany}-Week{week}.png", f"Week{week}/Min-{minCompany}.png")

Week0
Max Profit:$10.86 - SNPS
Min Profit:$-1.79 - SNA
Med Profit:$0.06 - KEY
Week1
Max Profit:$14.86 - POOL
Min Profit:$0.0 - DAL
Med Profit:$0.0 - DAL
Week2
Max Profit:$10.17 - PNC
Min Profit:$-10.54 - SNA
Med Profit:$0.0 - DAL
Week3
Max Profit:$200.0 - IVZ
Min Profit:$-0.19 - SNA
Med Profit:$0.0 - DAL
Week4
Max Profit:$10.71 - POOL
Min Profit:$-0.52 - SNA
Med Profit:$0.0 - SYF
Week5
Max Profit:$6.18 - TER
Min Profit:$0.0 - DAL
Med Profit:$0.0 - DAL
Week6
Max Profit:$3.97 - POOL
Min Profit:$-6.53 - SNA
Med Profit:$0.0 - DAL
Week7
Max Profit:$26.57 - IPGP
Min Profit:$-3.12 - SNA
Med Profit:$0.25 - IVZ


In [8]:
img = cv2.imread(f'Final/Week0/SNA-Week0.png')
text = pytesseract.image_to_string(img)
text = pytesseract.image_to_string(img)
text

'SNA Stock Price - Week 0\n\nzis | — Real stock price\n— Predicted stock price\n© Buy\n2164 a Sell- Profit\n¥ Sell- Loss\ng 2s\n22\n20\n\ndo 0s a0 a5 20 25 30 35 40\nProfit - ¢-1.79 Days\n'

In [9]:
profitC = float(regex.search("([$¢]\d+.\d+)|(\$0)|([$¢]-\d+.\d+)", text).group(0)[1:])
profitC

-1.79